In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BertModel
import pandas as pd
from torch.optim import Adam
from tqdm import tqdm 
import numpy as np

class IMDBDataset(Dataset):
    """
    Classe de dataset pour gérer les données textuelles de l'IMDB pour l'entraînement de modèles BERT.
    """

    def __init__(self, csv_file, device, model_name_or_path="bert-base-uncased", max_length=250):
        """
        Initialise le dataset avec les données du fichier CSV, le tokenizer BERT et le device (CPU ou GPU).

        Args:
            csv_file (str): Chemin du fichier CSV contenant les données.
            device (torch.device): Device pour le traitement des tensors.
            model_name_or_path (str): Nom ou chemin du modèle BERT pré-entraîné.
            max_length (int): Longueur maximale des séquences après tokenisation.
        """
        self.device = device
        self.df = pd.read_csv(csv_file)
        self.labels = self.df.sentiment.unique()
        labels_dict = {l: indx for indx, l in enumerate(self.labels)}
        self.df["sentiment"] = self.df["sentiment"].map(labels_dict)
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path) # On instancie également le tokenizer

    def __len__(self):
        """
        Retourne la taille du dataset.

        Returns:
            int: Nombre total d'exemples dans le dataset.
        """
        return len(self.df)

    def __getitem__(self, index):
        """
       on va parcourir par index et prendre l'élement corrspondant. l'element et le  text qui sera passer dans le tockenizee

        Args:
            index (int): Indice de l'exemple à récupérer.

        Returns:
            dict: Un dictionnaire contenant les input_ids, attention_mask et le label.
        """
        review_text = str(self.df.review[index])# on recupère les text et les index correspondants
        label_review = self.df.sentiment[index] # on recupère les 7 classes  et les index correspondants
        inputs = self.tokenizer(review_text, padding="max_length", max_length=self.max_length, truncation=True, return_tensors="pt")
        #  on a notre imnput et un effectue un  padding
        labels = torch.tensor(label_review) # on convertir les donner numpy en pytor. on a les labels

        return {
            "input_ids": inputs["input_ids"].squeeze(0).to(self.device),
            "attention_mask": inputs["attention_mask"].squeeze(0).to(self.device),
            "labels": labels.to(self.device)
        }

## fin castumer dataset qui va lire notre fichier csv




class CustomBert(nn.Module):
    """
    CustomBert est une classe de modèle utilisant BERT pour la classification de texte.
    Elle intègre un modèle BERT pré-entraîné et ajoute une couche de classification linéaire.
    """

    def __init__(self, model_name_or_path="bert-base-uncased", n_classes=2): ## nous avons 2 classe ou catégorie de classe
        """
        Initialise le modèle CustomBert avec un modèle BERT pré-entraîné et une couche de classification.

        Args:
            model_name_or_path (str): Nom ou chemin du modèle BERT pré-entraîné.
            n_classes (int): Nombre de classes pour la tâche de classification.
        """
        super(CustomBert, self).__init__()
        self.bert_pretrained = BertModel.from_pretrained(model_name_or_path)
        self.classifier = nn.Linear(self.bert_pretrained.config.hidden_size, n_classes)
        # # On veut faire une classification multiclase avec une couche linéaire

    def forward(self, input_ids, attention_mask):
        """
        Passe les entrées par le modèle BERT et la couche de classification.

        Args:
            input_ids (torch.Tensor): Les IDs d'entrée des tokens.
            attention_mask (torch.Tensor): Les masques d'attention.

        Returns:
            torch.Tensor: Les scores de classification pour chaque classe.
        """
        x = self.bert_pretrained(input_ids=input_ids, attention_mask=attention_mask)
        x = self.classifier(x.pooler_output)
        return x

  #def save_check_pointst(self, path):
  #  torch.save(self.tate_dic(), path)

def training_step(model, data_loader, loss_fn, optimiser):
    """
    Exécute une étape d'entraînement pour une époque complète.

    Args:
        model (nn.Module): Le modèle à entraîner.
        data_loader (DataLoader): Le DataLoader pour les données d'entraînement.
        loss_fn (nn.Module): La fonction de perte.
        optimiser (Optimizer): L'optimiseur pour mettre à jour les poids du modèle.

    Returns:
        float: La perte moyenne sur l'époque.
    """
    model.train()
    total_loss = 0

    for data in tqdm(data_loader, total=len(data_loader)):
        input_ids = data["input_ids"]
        attention_mask = data["attention_mask"]
        labels = data["labels"]

        output = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(output, labels)
        loss.backward()

        optimiser.step()
        optimiser.zero_grad()

        total_loss += loss.item()

    return total_loss / len(data_loader)# *100 multiplier par 100

def evaluation(model, test_dataloader, loss_fn):
    """
    Évalue le modèle sur les données de test.

    Args:
        model (nn.Module): Le modèle à évaluer.
        test_dataloader (DataLoader): Le DataLoader pour les données de test.
        loss_fn (nn.Module): La fonction de perte.

    Returns:
        tuple: La perte moyenne et l'exactitude sur le dataset de test.
    """
    model.eval()
    correct_predictions = 0
    losses = [] # a chaue fois que je fait une prédiction je recupère  la loss

    for data in tqdm(test_dataloader, total=len(test_dataloader)):
        input_ids = data["input_ids"]
        attention_mask = data["attention_mask"]
        labels = data["labels"]

        output = model(input_ids=input_ids, attention_mask=attention_mask)
        _, pred = output.max(1) #   je prend la classe prédict

        correct_predictions += torch.sum(pred == labels) # à chaque fois qu'on fait une prédiction on compare au labels
        loss = loss_fn(output, labels) # on calcule la loss
        losses.append(loss.item()) # on stock tous les loss dans notre liste "losses" grace à la methode append()

    return np.mean(losses), correct_predictions.double() / len(test_dataloader.dataset) # on calcul la moyenne de la loss




def main():
    """
    Fonction principale pour exécuter l'entraînement et l'évaluation du modèle.
    """
    print("Training::::")

    # Hyperparamètres et configuration
    N_EPOCHS =2
    LR = 2e-5
    BATCH_SIZE =8
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Chargement des datasets d'entraînement et de test
    train_dataset = IMDBDataset(csv_file="/content/train_dataset_allocine_french.csv", device=device, max_length=250)
    test_dataset  = IMDBDataset(csv_file="/content/test_dataset_allocine_french.csv", device=device, max_length=250)

    # Création des DataLoaders
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE)

    # Initialisation du modèle, de la fonction de perte et de l'optimiseur
    model = CustomBert()
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=LR)

    # Boucle d'entraînement et d'évaluation
    for epoch in range(N_EPOCHS):
        loss_train = training_step(model, train_dataloader, loss_fn, optimizer)
        loss_eval, accuracy = evaluation(model, test_dataloader, loss_fn)

        print(f"Train Loss: {loss_train} , Eval Loss: {loss_eval} , Accuracy: {accuracy}")

    # Sauvegarde  du modèle
    torch.save(model.state_dict(), "my_custom_bert_allocine.pth")

if __name__ == "__main__":
    main()


Training::::


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
100%|██████████| 250/250 [00:30<00:00,  8.30it/s]


Train Loss: 0.5978758344252905 , Eval Loss: 0.4227723827064037 , Accuracy: 0.8075


100%|██████████| 250/250 [00:30<00:00,  8.29it/s]


Train Loss: 0.3566832172026237 , Eval Loss: 0.3420039056092501 , Accuracy: 0.852


In [8]:

from transformers import AutoTokenizer, BertModel
import torch
import torch.nn as nn
import gradio as gr

# Définition de la classe CustomBert
class CustomBert(nn.Module):
    def __init__(self, model_name_or_path="bert-base-uncased", n_classes=2):
        super(CustomBert, self).__init__()
        self.bert_pretrained = BertModel.from_pretrained(model_name_or_path)
        self.classifier = nn.Linear(self.bert_pretrained.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        x = self.bert_pretrained(input_ids=input_ids, attention_mask=attention_mask)
        x = self.classifier(x.pooler_output)
        return x

# Initialisation du modèle
model = CustomBert()
model.load_state_dict(torch.load("my_custom_bert_allocine.pth"))
model.eval()

# Définition de la fonction de classification
def classifier_fn(texte: str):
    labels = {0:'positive', 1:'negative'}
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    inputs = tokenizer(texte, padding="max_length", max_length=512, truncation=True, return_tensors="pt")

    with torch.no_grad():
        output = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

    _, pred = output.max(1)
    return labels[pred.item()]

# Configuration et lancement de l'interface Gradio
demo = gr.Interface(
    fn=classifier_fn,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()



<ipython-input-8-3ce45ab12ca6>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("my_custom_bert_allocine.pth"))


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4ceb7d0d873957b7f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [7]:
! pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')